# Introduction

Here we extend the one-norm example.  Now, instead of considering a diamond, we consider a regular polygon in the xy-plane with `n=sides` sides.  (The variable `sides` is set at the start of the geometry section.)

We then connect the vertices of the polygon to (0, 0, 1) and (0, 0, -1) to make a "top" and then sample from a Gaussian conditional on the sample lying on that shape.  The default number of samples is 1000.  You made need to up the total number of samples to adequately explore the space depending on the structure of the anisotropic variance and the shape of the polygon.

# Import

In [ ]:
import os
import numpy as np
import importlib as il

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Make sure this is set to the repository base directory
os.getenv("PYTHONPATH", "")

In [ ]:
il.import_module("ctgauss")

In [ ]:
from ctgauss import IsotropicCTGauss, AnisotropicCTGauss

# Define geometry of subspace

In [ ]:
sides = 6

In [ ]:
theta = np.linspace(0, 2.*np.pi, sides+1)
# theta

In [ ]:
FXY = np.stack((-np.sin(theta[0:-1]), np.cos(theta[0:-1]), np.full((sides,), 0.0)), axis=1)
FXY

In [ ]:
FZ = np.array([[0, 0, 1]])
FZ

In [ ]:
F = np.vstack((FXY, FZ))
F

In [ ]:
g = np.zeros((sides+1,))
g

In [ ]:
L = np.zeros(((sides)*2, sides+1))

for j in range(2):
    for i in range(sides):
        reg_cur = i + j*sides
        reg_cc = (i + 1) % sides + j*sides
        reg_cw = (i - 1) % sides + j*sides
        reg_z  = i + ((j+1) % 2)*sides
        i_pos = i % sides
        i_neg = (i+1) % sides
        L[reg_cur, i_pos] = 1 * (reg_cw+1) # Fortran indexing
        L[reg_cur, i_neg] = -1 * (reg_cc+1) # Fortran indexing
        L[reg_cur, sides] = (-1)**j * (reg_z + 1) # Fortrain indexing
        
L

In [ ]:
theta_bisect = 0.5 * (theta[0:-1] + theta[1:])

In [ ]:
# Some precalc is necessary here to get the correct z value.  
# We want the distance between A[j] and (1, 0, 0) and (0, 0, 1) to be the same, which gives us psi.
# And then we want r to minimize those distances.
theta_bisect_0 = theta_bisect[0]
psi = np.arctan(np.cos(theta_bisect_0))
# rad = 2. / (np.tan(theta_bisect_0) + 1 + np.sin(psi))
rad = 1
theta_bisect_0, psi, rad
z_val = rad * np.sin(psi)

In [ ]:
Atop = np.stack((
    rad*np.cos(psi)*np.cos(theta_bisect), 
    rad*np.cos(psi)*np.sin(theta_bisect), 
    np.full((sides,), z_val)
), axis=1)
Abot = Atop.copy()
Abot[:,2] = -z_val

A = np.vstack((Atop, Abot)).reshape((sides*2, 3, 1))
A.squeeze()

In [ ]:
y = -np.full((2*sides, 1), fill_value=1.0)

# Examples

In [ ]:
F

In [ ]:
N = 1000
t_max = 0.5*np.pi
reg = 1
reg_c = (reg - 1)
A_squeezed = A.squeeze()
x0 = -y[reg_c] * A_squeezed[reg_c] / np.linalg.norm(A_squeezed[reg_c])**2
x0dot = np.array([-np.sin(theta_bisect[reg_c]), np.cos(theta_bisect[reg_c]), 0]) # Should be going to region 1 to 2

(x0, x0dot)

## Example 1 - Isotropic Variance

In [ ]:
mu = np.zeros((3,))
phi = 1.0

In [ ]:
rng = np.random.default_rng()

In [ ]:
ictg = IsotropicCTGauss(phi, mu, A, y, F, g, L)

In [ ]:
# DO WE PASS COTINUITY ERROR
ce = ictg.continuity_error()
np.all(ce < 1e15)

In [ ]:
(X1, X1dot, R1) = ictg.sample(rng, 2000, t_max, reg, x0, x0dot)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=35, azim=30)
ax.scatter3D(X1[:,0], X1[:,1], X1[:,2], c=R1, alpha=0.3, s=2)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title(f"$X \sim N(0, I_3)$ given $X$ on {sides}-sided top");

## Example 2 – Anisotropic Variance

In [ ]:
mu2 = np.zeros((3,))
Prec2 = np.array([
    [5.0, 0.0, 0.0],
    [0.0, 5.0, 0.0],
    [0.0, 0.0, 0.1]
])

In [ ]:
rng = np.random.default_rng()

In [ ]:
actg2 = AnisotropicCTGauss(Prec2, mu2, A, y, F, g, L, mean=True)

In [ ]:
# DO WE PASS COTINUITY ERROR
ce = actg2.continuity_error()
np.all(ce < 1e15)

In [ ]:
(X2, X2dot, R2) = actg2.sample(rng, N, t_max, reg, x0, x0dot)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=10, azim=20)
ax.scatter3D(X2[:,0], X2[:,1], X2[:,2], c=R2, alpha=0.4, s=2)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title(f"$X \sim N(0, D)$ given $X$ on {sides}-sided top,\n$D=diag(5., 5., 0.1)$");

## Example 3 – Anisotropic Variance

In [ ]:
mu3 = np.zeros((3,))
Prec3 = np.array([
    [0.01, 0.0, 0.0],
    [0.0, 0.01, 0.0],
    [0.0, 0.0, 100.0]
])

In [ ]:
rng = np.random.default_rng()

In [ ]:
actg3 = AnisotropicCTGauss(Prec3, mu3, A, y, F, g, L, mean=True)

In [ ]:
# DO WE PASS COTINUITY ERROR
ce = actg3.continuity_error()
np.all(ce < 1e15)

In [ ]:
(X3, X3dot, R3) = actg3.sample(rng, N, t_max, reg, x0, x0dot)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=10, azim=30)
ax.scatter3D(X3[:,0], X3[:,1], X3[:,2], c=R3, alpha=0.4, s=2)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title(f"$X \sim N(0, D)$ given $X$ on {sides}-sided top,\n$D=diag(0.1, 0.1, 100)$");

## Plot all together

In [ ]:
fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(1, 3, 1, projection='3d')
ax1.view_init(elev=35, azim=30)
ax1.scatter3D(X1[:,0], X1[:,1], X1[:,2], c=R1, alpha=0.3, s=2)
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_zlabel("z")
ax1.set_title(f"$X \sim N(0, I_3)$ given $X$ on {sides}-sided top");

ax2 = fig.add_subplot(1, 3, 2, projection='3d')
ax2.view_init(elev=10, azim=20)
ax2.scatter3D(X2[:,0], X2[:,1], X2[:,2], c=R2, alpha=0.4, s=2)
ax2.set_xlabel("x")
ax2.set_ylabel("y")
ax2.set_zlabel("z")
ax2.set_title(f"$X \sim N(0, D)$ given $X$ on {sides}-sided top,\n$D=diag(5., 5., 0.1)$");

ax3 = fig.add_subplot(1, 3, 3, projection='3d')
ax3.view_init(elev=10, azim=30)
ax3.scatter3D(X3[:,0], X3[:,1], X3[:,2], c=R3, alpha=0.4, s=2)
ax3.set_xlabel("x")
ax3.set_ylabel("y")
ax3.set_zlabel("z")
ax3.set_title(f"$X \sim N(0, D)$ given $X$ on {sides}-sided top,\n$D=diag(0.1, 0.1, 100)$");

# fig.savefig("ntop-example.png", dpi=300, pad_inches=0)